In [ ]:
# Hospital stay demographics

from db import run_query

with open("sql/hosp_demographics.sql", "r") as f:
	cohort = run_query(f.read())
cohort.head()

,subject_id,hadm_id,hosp_stay_num,pat_count,age,gender,insurance,hosp_los,dod,days_to_death,hospital_mortality
0,10000032,22595853,1,1,52.0,F,Medicaid,0.786111,2180-09-09,125.0,0
1,10000032,22841357,2,0,52.0,F,Medicaid,1.015278,2180-09-09,74.0,0
2,10000032,29079034,3,0,52.0,F,Medicaid,2.222222,2180-09-09,46.0,0
3,10000032,25742920,4,0,52.0,F,Medicaid,1.754167,2180-09-09,33.0,0
4,10000068,25022803,1,1,19.0,F,None,0.298611,None,NaN,0


In [ ]:
# Add 1 year mortality to table

cohort['one_year_mortality'] = cohort['days_to_death'].notnull().astype(int)
cohort.head()

,subject_id,hadm_id,hosp_stay_num,pat_count,age,gender,insurance,hosp_los,dod,days_to_death,hospital_mortality,one_year_mortality
0,10000032,22595853,1,1,52.0,F,Medicaid,0.786111,2180-09-09,125.0,0,1
1,10000032,22841357,2,0,52.0,F,Medicaid,1.015278,2180-09-09,74.0,0,1
2,10000032,29079034,3,0,52.0,F,Medicaid,2.222222,2180-09-09,46.0,0,1
3,10000032,25742920,4,0,52.0,F,Medicaid,1.754167,2180-09-09,33.0,0,1
4,10000068,25022803,1,1,19.0,F,None,0.298611,None,NaN,0,0


In [ ]:
# Days to death at last hospital stay

last_dod = cohort.groupby('subject_id')[['hosp_stay_num']].max().reset_index()
last_dod = last_dod.merge(cohort[['subject_id', 'hosp_stay_num', 'days_to_death']], on=['subject_id', 'hosp_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

last_dod.head()

,subject_id,hosp_stay_num,days_to_death_last_stay_id
0,10000032,4,33.0
1,10000068,1,NaN
2,10000084,2,47.0
3,10000108,1,NaN
4,10000117,2,NaN


In [ ]:
# Combine into hospital mortality table

cohort = cohort.merge(last_dod, how='left', on=['subject_id', 'hosp_stay_num'])
del last_dod
cohort.sort_values(['subject_id', 'hosp_stay_num'], inplace=True)

cohort.head()

,subject_id,hadm_id,hosp_stay_num,pat_count,age,gender,insurance,hosp_los,dod,days_to_death,hospital_mortality,one_year_mortality,days_to_death_last_stay_id_x,days_to_death_last_stay_id_y
0,10000032,22595853,1,1,52.0,F,Medicaid,0.786111,2180-09-09,125.0,0,1,NaN,NaN
1,10000032,22841357,2,0,52.0,F,Medicaid,1.015278,2180-09-09,74.0,0,1,NaN,NaN
2,10000032,29079034,3,0,52.0,F,Medicaid,2.222222,2180-09-09,46.0,0,1,NaN,NaN
3,10000032,25742920,4,0,52.0,F,Medicaid,1.754167,2180-09-09,33.0,0,1,33.0,33.0
4,10000068,25022803,1,1,19.0,F,None,0.298611,None,NaN,0,0,NaN,NaN


In [14]:
int_cols = cohort.dtypes.values=="Int64"
cohort.loc[:, int_cols] = cohort.loc[:, int_cols].astype(float)
cohort.loc[:, int_cols] = cohort.loc[:, int_cols].astype(int, errors="ignore")

cohort.info()
cohort.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546028 entries, 0 to 546027
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   subject_id                    546028 non-null  int64  
 1   hadm_id                       546028 non-null  int64  
 2   hosp_stay_num                 546028 non-null  int64  
 3   pat_count                     546028 non-null  int64  
 4   age                           546028 non-null  float64
 5   gender                        546028 non-null  object 
 6   insurance                     536673 non-null  object 
 7   hosp_los                      546028 non-null  float64
 8   dod                           144966 non-null  object 
 9   days_to_death                 144966 non-null  float64
 10  hospital_mortality            546028 non-null  int64  
 11  one_year_mortality            546028 non-null  int64  
 12  days_to_death_last_stay_id_x  36882 non-null

,subject_id,hadm_id,hosp_stay_num,pat_count,age,gender,insurance,hosp_los,dod,days_to_death,hospital_mortality,one_year_mortality,days_to_death_last_stay_id_x,days_to_death_last_stay_id_y
0,10000032,22595853,1,1,52.0,F,Medicaid,0.786111,2180-09-09,125.0,0,1,NaN,NaN
1,10000032,22841357,2,0,52.0,F,Medicaid,1.015278,2180-09-09,74.0,0,1,NaN,NaN
2,10000032,29079034,3,0,52.0,F,Medicaid,2.222222,2180-09-09,46.0,0,1,NaN,NaN
3,10000032,25742920,4,0,52.0,F,Medicaid,1.754167,2180-09-09,33.0,0,1,33.0,33.0
4,10000068,25022803,1,1,19.0,F,None,0.298611,None,NaN,0,0,NaN,NaN


In [1]:
# ICU stay demographics

from db import run_query

with open("sql/icu_demographics.sql", "r") as f:
	icu = run_query(f.read())

OperationalError: (psycopg2.errors.DiskFull) could not resize shared memory segment "/PostgreSQL.3940695134" to 16777216 bytes: No space left on device

[SQL: SELECT
      pat.subject_id,
      adm.hadm_id,
      icu.stay_id,
      ROW_NUMBER() OVER (PARTITION BY pat.subject_id ORDER BY icu.intime) AS icu_stay_num,
      DENSE_RANK() OVER (PARTITION BY pat.subject_id ORDER BY adm.admittime) AS hosp_stay_num,
      CASE
          WHEN FIRST_VALUE(icu.stay_id) OVER icustay_window = icu.stay_id THEN 1
          ELSE 0
      END AS pat_count,
      pat.anchor_age + (EXTRACT(YEAR FROM icu.intime) - pat.anchor_year) AS age,
      pat.gender,
      icu.first_careunit,
      icu.los AS icu_los,
      EXTRACT(EPOCH FROM (adm.dischtime - adm.admittime)) / 3600 / 24 AS hosp_los,
      pat.dod,
      DATE(pat.dod) - DATE(adm.dischtime) AS days_to_death,
      CASE WHEN DATE(pat.dod) - DATE(adm.dischtime) = 0 THEN 1 ELSE 0 END AS hospital_mortality,
      CASE WHEN DATE(pat.dod) - DATE(icu.outtime)

In [ ]:
# Add 1 year mortality
icu['one_year_mortality'] = icu['days_to_death'].notnull().astype(int)

# Days to death at last ICU stay
last_dod = icu.groupby('subject_id')[['icu_stay_num']].max().reset_index()
last_dod = last_dod.merge(icu[['subject_id', 'icu_stay_num', 'days_to_death']], on=['subject_id', 'icu_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

icu = icu.merge(last_dod, how='left', on=['subject_id', 'icu_stay_num'])
del last_dod
icu.sort_values(['subject_id', 'icu_stay_num'], inplace=True)

# add a grouping variable for table one so we can have hospital mortality as a group and a row
icu['hosp_mort'] = icu  ['hospital_mortality']

# fix some data type issues
int_cols = icu.dtypes.values=="Int64"
icu.loc[:, int_cols] = icu.loc[:, int_cols].astype(float)
icu.loc[:, int_cols] = icu.loc[:, int_cols].astype(int, errors="ignore")